In [2]:
import os
from tqdm import tqdm
import pickle
import random
from typing import List, Dict, Tuple
from anndata import AnnData
import time
import os

import pandas as pd
import numpy as np
import math
import scipy
from sklearn.metrics.pairwise import pairwise_kernels
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels

from molmass import Formula
import metaspace
import linex2metaspace as lx2m
from scipy.cluster.hierarchy import linkage
from sklearn.cluster import AgglomerativeClustering
from scipy import stats
from statannotations.Annotator import Annotator
import networkx as nx

Package 'lynx' (LipidLynxX) not available. Lipid name conversions will not be possible.


In [3]:
import utils
from coloc_utils import *
from config import store_dir, data_dir, date_key, enrichment_dir
%load_ext autoreload
%autoreload 2

# Load datasets

In [10]:
pos_lip_top_datasets = pickle.load(open(os.path.join(store_dir, 'pos_lip_top_datasets_list.pickle'), "rb" ))
tissue_ads = load_alltissue_datasets(pos_lip_top_datasets)

# exclude datasets
del tissue_ads['Brain']['2022-08-24_00h20m06s']
del tissue_ads['Brain']['2022-08-23_23h48m59s']
del tissue_ads['Buccal mucosa']

# Calculate mass of every molecule
for tis, dsd in tissue_ads.items():
    for dsid, ds in dsd.items():
        tissue_ads[tis][dsid].var['mass'] = [Formula(x).mass for x in tissue_ads[tis][dsid].var['formula'].values]

# Filter by mass
for tis, dsd in tissue_ads.items():
    for dsid, ds in dsd.items():
        tmp = tissue_ads[tis][dsid]
        tmp = tmp[:, tmp.var['mass'] <= 900]
        tissue_ads[tis][dsid] = tmp[:, tmp.var['mass'] >= 400]

# Remove isobars
tissue_ads = mark_isobars(tissue_ads, ppm_threshold=3)
ll = []
for tis, dsd in tissue_ads.items():
    for dsid, ds in dsd.items():
        if ds[:, ~ds.var['has_isobar']].shape[1] < 10:
            ll.append((tis, dsid))
        else:
            # Remove too big datasets
            if ds.shape[0] > 100000:
                ll.append((tis, dsid))
for item in ll:
    del tissue_ads[item[0]][item[1]]

/tmp/ipykernel_781/428607169.py:12: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  tissue_ads[tis][dsid].var['mass'] = [Formula(x).mass for x in tissue_ads[tis][dsid].var['formula'].values]
/tmp/ipykernel_781/428607169.py:12: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  tissue_ads[tis][dsid].var['mass'] = [Formula(x).mass for x in tissue_ads[tis][dsid].var['formula'].values]
/tmp/ipykernel_781/428607169.py:12: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  tissue_ads[tis][dsid].var['mass'] = [Formula(x).mass for x in tissue_ads[tis][dsid].var['formula'].values]
/tmp/ipykernel_781/428607169.py:12: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  tissue_ads[tis][dsid].var['mass'] = [Formula(x).mass for x in tissue_ads[tis][dsid].var['formula'].values]
/tmp/ipykernel_7

### Analysis only for brain datasets

In [11]:
# Make temporary folders for misty results

misty_base_path = '/scratch/trose/misty'

data_path = os.path.join(misty_base_path, 'data')
results_path = os.path.join(misty_base_path, 'results')
log_path = os.path.join(misty_base_path, 'log')

if not os.path.isdir(misty_base_path):
    os.mkdir(misty_base_path)
    
if not os.path.isdir(data_path):
    os.mkdir(data_path)
    
if not os.path.isdir(results_path):
    os.mkdir(results_path)
    
if not os.path.isdir(log_path):
    os.mkdir(log_path)

In [14]:
ds_list = []

for tis in ['Brain']:
    print(f'** {tis} **')
    for ds in tissue_ads[tis].keys():
        tmp = tissue_ads[tis][ds].uns['metaspace_metadata']['MS_Analysis']
        if 'Pixel_Size' in tmp.keys() and tmp['Pixel_Size']['Xaxis'] <= 50 and tmp['Pixel_Size']['Yaxis'] <= 50:
            adat = tissue_ads[tis][ds]
            unique_labels = np.unique(adat.var.formula)
            sums = {}

            # Iterate over the unique labels
            for label in unique_labels:
                # Get the indices of rows with the current label
                indices = np.where(adat.var.formula == label)[0]
                # Sum up the corresponding rows and store the result
                if len(indices)>1:
                    sums[label] = np.sum(adat.X[:, indices], axis=1)
                else:
                    sums[label] = adat.X[:, indices[0]]

            tmp_array = np.stack(list(sums.values()))
            tmp_molecules = np.array(list(sums.keys()))

            tmp_ymax = adat.obs['y'].max()+1
            
            # Coloc preprocessing:
            conv_data = utils.coloc_preprocessing_array(tmp_array.transpose(), tmp_ymax)

            df = pd.DataFrame(conv_data.transpose(), columns=tmp_molecules)
            df['row'] = adat.obs.reset_index()['x'] * adat.uns['metaspace_metadata']['MS_Analysis']['Pixel_Size']['Xaxis']
            df['col'] = adat.obs.reset_index()['y'] * adat.uns['metaspace_metadata']['MS_Analysis']['Pixel_Size']['Yaxis']

            df = df.loc[~(df.drop(columns=['row', 'col']).sum(axis=1) == 0), :]
            df = df.loc[:, df.sum()!=0]
            
            curr_ds_path = os.path.join(data_path, f'{tis}_{ds}.csv')
            ds_list.append(curr_ds_path)
            df.to_csv(curr_ds_path)

** Brain **


# Create slurm file

In [16]:
log_path

'/scratch/trose/misty/log'

In [25]:
files = '"'+'" "'.join(ds_list)+'"'

In [33]:
files = '"'+'" "'.join(ds_list)+'"'
slurm_file = f"""#!/bin/bash
#SBATCH -J mistypl
#SBATCH -A alexandr                # group to which you belong
#SBATCH -N 1                        # number of nodes
#SBATCH -n 4                        # number of cores
#SBATCH --mem 100G                    # memory pool for all cores
#SBATCH -t 0-11:00:00                   # runtime limit (D-HH:MM:SS)
#SBATCH --array=1-{len(ds_list)}  # Adjust the range according to your datasets
#SBATCH --output={log_path}/output_%A_%a.txt
#SBATCH --error={log_path}/error_%A_%a.txt
#SBATCH --mail-type=END,FAIL        # notifications for job done & fail
#SBATCH --mail-user=tim.rose@embl.de # send-to address

module load R/4.2.2-foss-2022b
datasets=({files})

# Get the dataset name for this job
dataset_name="${{datasets[$SLURM_ARRAY_TASK_ID - 1]}}"

Rscript misty_run.R $dataset_name {results_path}

"""
output_file_path = "scripts/misty_run.sh"

# Write the slurm_file content to the file
with open(output_file_path, "w") as f:
    f.write(slurm_file)
f.close()

In [32]:
slurm_file

'\n#!/bin/bash\n#SBATCH -J mistypl\n#SBATCH -A alexandr                # group to which you belong\n#SBATCH -N 1                        # number of nodes\n#SBATCH -n 4                        # number of cores\n#SBATCH --mem 100G                    # memory pool for all cores\n#SBATCH -t 0-11:00:00                   # runtime limit (D-HH:MM:SS)\n#SBATCH --array=1-5  # Adjust the range according to your datasets\n#SBATCH --output=/scratch/trose/misty/log/output_%A_%a.txt\n#SBATCH --error=/scratch/trose/misty/log/error_%A_%a.txt\n#SBATCH --mail-type=END,FAIL        # notifications for job done & fail\n#SBATCH --mail-user=tim.rose@embl.de # send-to address\n\nmodule load R/4.2.2-foss-2022b\ndatasets=("/scratch/trose/misty/data/Brain_2022-05-31_10h46m34s.csv" "/scratch/trose/misty/data/Brain_2022-05-31_10h27m17s.csv" "/scratch/trose/misty/data/Brain_2022-05-30_20h44m19s.csv" "/scratch/trose/misty/data/Brain_2021-11-11_11h49m37s.csv" "/scratch/trose/misty/data/Brain_2020-05-19_21h56m17s.csv"